In [1]:
import numpy as np
import cv2
from LOBOROBOT import LOBOROBOT # 载入机器人库
from gpiozero import Button
from gpiozero import LED
import time 
import threading # 线程W
import ctypes
import inspect
import time

In [2]:
#超声模块依赖
from gpiozero import DistanceSensor
from time import sleep

In [3]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [4]:
# 按键及LED初始化
Btn  = Button(19,pull_up=True)   # 按键端口
Gpin = LED(5)
Rpin = LED(6)

#SensorRight = Button(16,pull_up=True)     # 右侧红外避障传感器
#SensorLeft  = Button(12,pull_up=True)    # 左侧红外避障传感器

#初始化舵机
clbrobot = LOBOROBOT()  # 实例化机器人对象
clbrobot.t_stop(0)  # 停止

# Configure min and max servo pulse lengths
servo_min = 150  # Min pulse length out of 4096
servo_max = 600  # Max pulse length out of 4096

# 频率设置为50hz，适用于舵机系统。
clbrobot.set_servo_angle(10,85)  # 底座舵机 90 
clbrobot.set_servo_angle(9,25)  # 顶部舵机 145(9,50)
time.sleep(0.5)

In [5]:
# 按键标志位
keyflag = 0

# 按键控制函数
def keysacn():
    global keyflag
    print('*****************************************') 
    print('* makerobo Button Pressed!*') 
    print('*****************************************')
    Rpin.on()  # 打开红色LED
    Gpin.off() # 关闭绿色LED
    keyflag = 1   # 按键标志位置1

def released():
    print("button was released")
    Rpin.off()   # 关闭红色LED
    Gpin.on()    # 打开绿色LED

# 按键中断函数+
Btn.when_pressed = keysacn
Btn.when_released = released

In [6]:
# 载入显示库
import ipywidgets.widgets as widgets
from IPython.display import display
import libcamera
from picamera2 import Picamera2

In [7]:
# 云台摄像头可视化
image = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image)

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

Image(value=b'', format='jpeg', height='120', width='180')

In [8]:
# # 云台摄像头可视化
# image_gray_0 = widgets.Image(format='jpeg', width=180, height=120)
# # image = widgets.Image(format='jpeg', width=1920, height=1080)
# display(image_gray_0)

In [9]:
# # 云台摄像头可视化
# image_blur_0 = widgets.Image(format='jpeg', width=180, height=120)
# # image = widgets.Image(format='jpeg', width=1920, height=1080)
# display(image_blur_0)

In [10]:
# 云台摄像头可视化
image_thresh1_0 = widgets.Image(format='jpeg', width=180, height=120)
# image = widgets.Image(format='jpeg', width=1920, height=1080)
display(image_thresh1_0)

Image(value=b'', format='jpeg', height='120', width='180')

In [11]:
# # 云台摄像头可视化
# image_mask_0 = widgets.Image(format='jpeg', width=180, height=120)
# # image = widgets.Image(format='jpeg', width=1920, height=1080)
# display(image_mask_0)

In [12]:
# 识别摄像头可视化
image_1 = widgets.Image(format='jpeg', width=1080, height=720)
display(image_1)

Image(value=b'', format='jpeg', height='720', width='1080')

In [13]:
def get_camera():
    # 获取第一个摄像头
    picamera_0 = Picamera2(camera_num=0)
    config = picamera_0.create_preview_configuration(main={"format": 'RGB888', "size": (180, 120)},
                                               raw={"format": "SRGGB12", "size": (720, 480)})
    config["transform"] = libcamera.Transform(hflip=1, vflip=1)
    picamera_0.configure(config)
    picamera_0.start()
    
    # 获取第二个摄像头
    picamera_1 = Picamera2(camera_num=1)
    config = picamera_1.create_preview_configuration(main={"format": 'RGB888', "size": (1280, 720)},
                                                   raw={"format": "SRGGB12", "size": (1280, 720)})
    config["transform"] = libcamera.Transform(hflip=0, vflip=1)
    picamera_1.configure(config)
    picamera_1.start()
    return picamera_0,picamera_1

In [14]:
from ultralytics import YOLO
import gc
import pygame,time,os

# def Video_display_origin():
#     picamera_0, picamera_1 = get_camera()
#     no_line_counter = 0  # 连续丢线帧计数器
#     frame_counter = 0  # 抽帧计数
#     _already_played = False
#     makerobo_sensor = DistanceSensor(echo=21, trigger=20, max_distance=3, threshold_distance=0.2) #超声模块
#     while True:
#         frame_counter += 1
#         dis = makerobo_sensor.distance * 100  # 超声模块测量距离值，并把m单位换成cm单位
#         no_line_counter = line_track(picamera_0, no_line_counter, keyflag)
#         if dis <=20 and not _already_played:   # 上升沿触发
#             clbrobot.t_stop(3)
#             _already_played = True             # 标记“已播”
#             results = yolo_detect(picamera_1, frame_counter)
#             for result in results:
#                 boxes = result.boxes  # Boxes 对象
#                 names = result.names  # 类别字典

#             if boxes is not None:
#                 for cls in boxes.cls:
#                     print("识别种类：", names[int(cls)])
#                     if names[int(cls)] == "仙人球":
#                         detect_result_read('results_read/仙人掌.mp3')
#                     elif names[int(cls)] == "薄荷叶":
#                         detect_result_read('results_read/薄荷叶.mp3')
#                     elif names[int(cls)] == "金枝玉叶":
#                         detect_result_read('results_read/金枝玉叶.mp3')
#                     elif names[int(cls)] == "吉利红":
#                         detect_result_read('results_read/吉丽红.mp3')
#                     elif names[int(cls)] == "绿萝":
#                         detect_result_read('results_read/绿箩.mp3')
#         elif dis >20:                       # 一旦回到 0 就解除锁定
#             _already_played = False



def Video_display():
    picamera_0, picamera_1 = get_camera()
    no_line_counter = 0  # 连续丢线帧计数器
    frame_counter = 0    # 抽帧计数
    _already_played = False
    last_play_time = 0   # 上一次 detect_result_read 的时间戳
    makerobo_sensor = DistanceSensor(echo=21, trigger=20, max_distance=3, threshold_distance=0.2) #超声模块
    while True:
        dis = makerobo_sensor.distance * 100  # 超声模块测量距离值，并把m单位换成cm单位
        frame_counter += 1
        no_line_counter = line_track(picamera_0, no_line_counter, keyflag)

        # --- 检测触发 ---
        if dis <=20 and not _already_played:
            clbrobot.t_stop(3)
            _already_played = True
            last_play_time = time.time()  # 记录当前时间
            results = yolo_detect(picamera_1, frame_counter)

            for result in results:
                boxes = result.boxes  # Boxes 对象
                names = result.names  # 类别字典

            if boxes is not None:
                for cls in boxes.cls:
                    name = names[int(cls)]
                    print("识别种类：", name)
                    if name == "仙人球":
                        detect_result_read('results_read/仙人掌.mp3')
                    elif name == "薄荷叶":
                        detect_result_read('results_read/薄荷叶.mp3')
                    elif name == "金枝玉叶":
                        detect_result_read('results_read/金枝玉叶.mp3')
                    elif name == "吉利红":
                        detect_result_read('results_read/吉丽红.mp3')
                    elif name == "绿萝":
                        detect_result_read('results_read/绿箩.mp3')

        # --- 解锁条件 ---
        elif dis >20:
            # 仅在播放完毕且超过5秒后才能重新触发
            if _already_played and (time.time() - last_play_time >= 5):
                _already_played = False

            
        

def yolo_detect(picamera_1, frame_counter):
    # Load a pretrained YOLO11n model
    model = YOLO("best_ncnn_model")
    frame_1 = picamera_1.capture_array()
    if frame_counter >= 1:
        results = model.predict(frame_1)  # 只在这里用模型推理
        #print("识别结果为：", results)
        '''
        # 输出识别结果
        for result in results:
            boxes = result.boxes  # Boxes 对象
            names = result.names  # 类别字典

            if boxes is not None:
                for cls in boxes.cls:
                    print("识别种类：", names[int(cls)])
        '''
        # 绘制检测框
        annotated_frame = results[0].plot()  # 返回带框的图像
        # 显示或赋值给你原先的 widgets.Image
        image_1.value = bgr8_to_jpeg(annotated_frame)
        frame_counter = 0  # 计数器归零
    else:
        image_1.value = bgr8_to_jpeg(frame_1)
    return results

def detect_result_read(mp3):
    os.environ['SDL_AUDIODRIVER'] = 'alsa'
    os.environ['AUDIODEV'] = 'hw:2,0'
    pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)
    pygame.mixer.music.load(mp3)
    pygame.mixer.music.play()

def line_track(picamera_0, no_line_counter, keyflag):
    if keyflag == 1:
        frame_0 = picamera_0.capture_array()
        #frame_0 = cv2.flip(frame_0, 1)
        #print("Height:", frame_0.shape[0])
        #print("Width :", frame_0.shape[1])
        #crop_img_0 = frame_0[0:180, 0:120]
        crop_img_0 = frame_0
        # gray_0 = cv2.cvtColor(crop_img_0, cv2.COLOR_BGR2GRAY)
        # blur_0 = cv2.GaussianBlur(gray_0, (5,5), 0)
        # ret_0, thresh1_0 = cv2.threshold(blur_0, 60, 255, cv2.THRESH_BINARY_INV)
        hsv = cv2.cvtColor(crop_img_0, cv2.COLOR_BGR2HSV)

        # 检测黑色
        lower_black = np.array([0, 0, 0])
        upper_black = np.array([180, 255, 60])
        thresh1_0 = cv2.inRange(hsv, lower_black, upper_black)
        
        mask_1 = cv2.erode(thresh1_0, None, iterations=2)
        mask_0 = cv2.dilate(mask_1, None, iterations=2)
        contours_0, hierarchy_0 = cv2.findContours(mask_0.copy(), 1, cv2.CHAIN_APPROX_NONE)
        #print("len(contours_0): ",len(contours_0))
        if len(contours_0) > 0:
            no_line_counter = 0  # 重置丢线计数
            c = max(contours_0, key=cv2.contourArea)
            M = cv2.moments(c)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])

                cv2.line(crop_img_0, (cx, 0), (cx, 120), (0, 0, 255), 2)
                cv2.line(crop_img_0, (120, 0), (120, 120), (0, 255, 255), 2)
                cv2.line(crop_img_0, (70, 0), (70, 120), (0, 255, 255), 2)
                
                cv2.line(crop_img_0, (0, 60), (180, 60), (255, 0, 0), 2)
                cv2.line(crop_img_0, (0, cy), (180, cy), (255, 255, 0), 2)
                
                cv2.drawContours(crop_img_0, contours_0, -1, (0, 255, 0), 1)
                #print("cx:",cx)
                #if cx >= 115:
                if cx >= 120:
                    clbrobot.turnRight(25, 0)
                    # clbrobot.turnRight(25, 0)
                    #print("➡ Turn Right!")
                elif cx < 120 and cx > 70:
                    clbrobot.t_up(30, 0)
                    #print("⬆ On Track!")
                elif cx <= 70:
                    clbrobot.turnLeft(25, 0)
                    # clbrobot.turnLeft(25, 0)
                    #print("⬅ Turn Left!")
            else:
                print("异常")
        else:
            no_line_counter += 1
            print(f"⚠ Line lost {no_line_counter} frame(s)")
            if no_line_counter >= 6:  # 连续6帧未检测到线才停车
                clbrobot.t_down(30,1.5)
                no_line_counter = 0
                print("🛑 Stop due to no line detected!")
        #print("cx:",cx,"len(contours_0): ",len(contours_0))
        #image.value = bgr8_to_jpeg(crop_img_0)
        image.value = bgr8_to_jpeg(frame_0)
        #image_gray_0.value = bgr8_to_jpeg(gray_0)
        #image_blur_0.value = bgr8_to_jpeg(blur_0)
        image_thresh1_0.value = bgr8_to_jpeg(thresh1_0)
        #image_mask_1.value = bgr8_to_jpeg(mask_1)
        #image_mask_0.value = bgr8_to_jpeg(mask_0)
        return no_line_counter

pygame 2.1.2 (SDL 2.26.5, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

/tmp/ipykernel_2508/3481196328.py:2: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  t.setDaemon(True)
[0:17:34.490061718] [2572]  INFO Camera camera_manager.cpp:325 libcamera v0.3.2+27-7330f29b
[0:17:34.498911298] [2573]  INFO RPI pisp.cpp:695 libpisp version v1.0.7 28196ed6edcf 29-08-2024 (16:33:32)
[0:17:34.508283911] [2573]  INFO RPI pisp.cpp:1154 Registered camera /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 to CFE device /dev/media1 and ISP device /dev/media0 using PiSP variant BCM2712_D0
[0:17:34.624703926] [2573]  WARN V4L2 v4l2_pixelformat.cpp:346 Unsupported V4L2 pixel format H264
[0:17:34.629268821] [2572]  INFO Camera camera.cpp:1197 configuring streams: (0) 180x120-RGB888 (1) 1920x1080-SGRBG16
[0:17:34.629378168] [2573]  INFO RPI pisp.cpp:1450 Sensor: /base/axi/pcie@120000/rp1/i2c@80000/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected CFE format: 1920x1080-GR16
[0:17:34.751403372] [2572]  INFO Camera camera.cpp:1197 c

In [16]:
stop_thread(t)
clbrobot.t_stop(0)    

In [ ]:
pwd